In [7]:
import pandas as pd

In [8]:
import nltk
from nltk.corpus import stopwords

In [9]:
import string
string.punctuation
l1=string.punctuation
l2=stopwords.words("english")

In [10]:
from sklearn.feature_extraction.text import CountVectorizer

In [11]:
ams=pd.read_csv("AmazonFoodReviews.csv")

In [12]:
ams.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [13]:
ams=ams.drop('ProductId',axis=1)
ams=ams.drop('Id',axis=1)
ams=ams.drop('UserId',axis=1)
ams=ams.drop('ProfileName',axis=1)
ams=ams.drop('HelpfulnessNumerator',axis=1)
ams=ams.drop('HelpfulnessDenominator',axis=1)
ams=ams.drop('Time',axis=1)
ams=ams.drop('Summary',axis=1)

In [14]:
ams.head()

,Score,Text
0,5,I have bought several of the Vitality canned d...
1,1,Product arrived labeled as Jumbo Salted Peanut...
2,4,This is a confection that has been around a fe...
3,2,If you are looking for the secret ingredient i...
4,5,Great taffy at a great price. There was a wid...


In [15]:
ams.rename(columns={'Score':'Y','Text':'X'},inplace=True)

In [16]:
ams.Y.replace({2:1,3:2,4:2,5:2},inplace=True)# replace 1 and 2 with 1 as it is negative 3 ton 2 as neutral and 4 and 5 to 3 as psoitive

C:\Users\HP\AppData\Local\Temp\ipykernel_13188\4068003695.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  ams.Y.replace({2:1,3:2,4:2,5:2},inplace=True)# replace 1 and 2 with 1 as it is negative 3 ton 2 as neutral and 4 and 5 to 3 as psoitive


In [17]:
ams.head()

,Y,X
0,2,I have bought several of the Vitality canned d...
1,1,Product arrived labeled as Jumbo Salted Peanut...
2,2,This is a confection that has been around a fe...
3,1,If you are looking for the secret ingredient i...
4,2,Great taffy at a great price. There was a wid...


In [18]:
ams['X']=ams['X'].str.lower()

In [19]:
import nltk
from nltk.corpus import stopwords

In [20]:
import string
string.punctuation
l1=string.punctuation
l2=stopwords.words("english")

In [21]:
def text_process(mess):            ### creating a function

    """                                                        ## a docstring

    1. remove the punctuation

    2. remove the stopwords

    3. return the list of clean textwords

    """
    nopunc = [char for char in mess if char not in l1]

    nopunc = "".join(nopunc)

    return [ word for word in nopunc.split() if word not in l2]

In [22]:
bow_transformer=CountVectorizer(analyzer=text_process).fit(ams['X'])

In [23]:
ams_bow=bow_transformer.transform(ams.X)

In [24]:
from sklearn.model_selection import train_test_split

In [25]:
train1_x, test1_x, train1_y, test1_y = train_test_split(ams_bow, ams.Y,test_size=.2)

In [26]:
from sklearn.linear_model import LogisticRegression        
logreg=LogisticRegression()
logreg.fit(train1_x,train1_y)

C:\Users\HP\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [27]:
pred_log = logreg.predict(test1_x)
pred_log

array([2, 2, 1, ..., 2, 2, 1], dtype=int64)

In [28]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, recall_score,f1_score

In [29]:
tab1 = confusion_matrix(test1_y,pred_log)
tab1

array([[10937,  5521],
       [ 2515, 94718]], dtype=int64)

In [30]:
print(classification_report(test1_y,pred_log))

              precision    recall  f1-score   support

           1       0.81      0.66      0.73     16458
           2       0.94      0.97      0.96     97233

    accuracy                           0.93    113691
   macro avg       0.88      0.82      0.85    113691
weighted avg       0.93      0.93      0.93    113691



In [31]:
from sklearn.tree import DecisionTreeClassifier
dt =DecisionTreeClassifier() #this simple is on gini---maxdepth , min split,best result to check no of shape in x or run the it multiple tinmes
dt.fit(train1_x ,train1_y)

DecisionTreeClassifier()

In [32]:
pred_dt=dt.predict(test1_x)
pred_dt

array([2, 2, 1, ..., 2, 2, 1], dtype=int64)

In [33]:
tab1 = confusion_matrix(test1_y, pred_dt)#apart from diagonal all are error
tab1

array([[10759,  5699],
       [ 4460, 92773]], dtype=int64)

In [34]:
print(classification_report(test1_y, pred_dt))

              precision    recall  f1-score   support

           1       0.71      0.65      0.68     16458
           2       0.94      0.95      0.95     97233

    accuracy                           0.91    113691
   macro avg       0.82      0.80      0.81    113691
weighted avg       0.91      0.91      0.91    113691



In [35]:
from sklearn.naive_bayes import MultinomialNB

nb = MultinomialNB()
nb.fit(train1_x,train1_y)

MultinomialNB()

In [36]:
pred = nb.predict(test1_x)
pred

array([2, 2, 1, ..., 1, 2, 1], dtype=int64)

In [37]:
tab = confusion_matrix(test1_y,pred)
tab

array([[10681,  5777],
       [ 5027, 92206]], dtype=int64)

In [38]:
print(classification_report(test1_y,pred))

              precision    recall  f1-score   support

           1       0.68      0.65      0.66     16458
           2       0.94      0.95      0.94     97233

    accuracy                           0.90    113691
   macro avg       0.81      0.80      0.80    113691
weighted avg       0.90      0.90      0.90    113691

